In [14]:
from __future__ import print_function
import numpy as np

# Asymptotic performance of probabilistic PBT with EPR pairs

Compute the probability of success for probabilistic port-based teleportation using $N$ ports with local dimension $d$, as well as the asymptotic behavior.

In [15]:
def specht(mu):
    """Dimension of Specht module [mu]. Denoted d_mu in [CLM+18]."""
    return StandardTableaux(mu).cardinality()

def weyl(d, mu):
    """Dimension of Weyl module V_mu^d. Denoted m_{d,mu} in [CLM+18]."""
    return SemistandardTableaux(shape=mu, max_entry=d).cardinality()

def p_EPR(d, N):
    """
    Exact expression for the success probability from [SSMH17]. Denoted p^EPR_d(N) in [CLM+18].
    We use the formula derived in the proof of Theorem 1.3 of [CLM+18].
    """
    return d**-N * sum(
            (weyl(d, alpha) * specht(alpha) * N / (alpha[0] + d)).n()
        for alpha in Partitions(n=N-1, max_length=d))

p_EPR(2, 10)

0.598721590909091

In [16]:
def GUE_0_lambda_max(d):
    A = np.random.randn(d, d) + 1j * np.random.randn(d, d)
    A = (A + A.T.conj()) / 2
    A = A - A.trace() / d * np.eye(d)
    return np.sort(np.linalg.eigvalsh(A))[-1]

def GUE_0_lambda_max_estimate(d, num_samples):
    """
    In [CLM+18], we prove that p^EPR_d(N) = 1 - c*sqrt(d/(N-1)) + o(1/sqrt(N)), where c = E[lambda_max(G)].
    This function estimates the coefficient c.
    """
    return np.mean([GUE_0_lambda_max(d) for _ in range(num_samples)])

Compare with the analytical answer for d=2.

In [17]:
%%time
got = GUE_0_lambda_max_estimate(2, 10000)
expected = np.sqrt(4 / np.pi)
print(abs(got - expected))

0.004373230585100751
CPU times: user 1.01 s, sys: 3.67 ms, total: 1.01 s
Wall time: 1.01 s


In [18]:
def print_table(d, N_min, N_max, N_step=10, file=sys.stdout, debug=False):
    print('N p FO', file=file)
    for N in range(N_min, N_max+1, N_step):
        p = p_EPR(d, N)
        FO = np.sqrt((N-1) / d) * (1 - p)
        print(N, p, FO, file=file)
        file.flush()
        if debug:
            sys.stdout.flush()
            print(N, p, FO, file=sys.stderr)
            sys.stderr.flush()

print_table(d=2, N_min=10, N_max=250)

N p FO
10 0.598721590909091 0.8512400526357939
20 0.695224943615141 0.939379812667241
30 0.743329168327393 0.9773734084559812
40 0.773648868663281 0.999539531894337
50 0.795057497818821 1.0144136313195546
60 0.811237096605812 1.02524499222635
70 0.824033559961178 1.0335688108141619
80 0.834489921464531 1.0402147138087754
90 0.843246765011590 1.0456745010902115
100 0.850722515324652 1.0502602000620773
110 0.857203792605575 1.0541803679255395
120 0.862894504989684 1.05758027938328
130 0.867944266402691 1.0605645817426965
140 0.872465709484295 1.063210783610396
150 0.876545487584933 1.0655776552440097
160 0.880251518550620 1.0677106608767604
170 0.883637900401164 1.069645586092736
180 0.886748335296968 1.071411027776788
190 0.889618569355172 1.0730301448926276
200 0.892278166359596 1.0745219158135655
210 0.894751820338421 1.0759020583477579
220 0.897060342438345 1.0771837142895937
230 0.899221413579917 1.0783779664640107
240 0.901250165934643 1.0794942345785397
250 0.903159637450401 1.080

Generate data files for the plots in [CLM+18].

In [12]:
%%time
!mkdir -p out

NUM_SAMPLES = 100000

for d in [3, 4, 5]:
    print("%d %.03f" % (d, GUE_0_lambda_max_estimate(d, NUM_SAMPLES)))

3 1.905
4 2.527
5 3.064
CPU times: user 31.6 s, sys: 170 ms, total: 31.8 s
Wall time: 32.7 s


In [ ]:
%%time
!mkdir -p out

N_min = 10
N_max = 500

for d in [2, 3, 4, 5]:
    print('# d = %s, N = %s..%s' % (d, N_min, N_max))

    fname = 'out/pbt-prob-epr-d%d.txt' % d
    with open(fname, 'w') as f:
        print_table(d, N_min, N_max, file=f) #, debug=True)
    print()

print('Done.')

# d = 2, N = 10..500

# d = 3, N = 10..500

# d = 4, N = 10..500

# d = 5, N = 10..500


### References

[CLM+18] M. Christandl, F. Leditzky, C. Majenz, G. Smith, F. Speelman, M. Walter, "Asymptotic performance of port-based teleportation", 2018. [arXiv:1809.10751](https://arxiv.org/abs/1809.10751)

[SSMH17] M. Studzinski, S. Strelchuk, M. Mozrzymas, M. Horodecki, "Port-based teleportation in arbitrary dimension", Scientific Reports 7 (2017). [arXiv:1612.09260](https://arxiv.org/abs/1612.09260)